In [1]:
import PyPDF2

from langchain_community.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama

from IPython.display import display, Markdown, clear_output
import time


/Users/christopheatten/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Initialize the LLaMA model via Ollama
model_id = "llama3.1"
model = Ollama(model=model_id)

In [3]:
# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in range(len(reader.pages)):
            text += reader.pages[page].extract_text()
    return text

In [50]:
def generate_refined_summary(model, text, max_iterations=5):
    iteration = 0
    current_summary = None
    questions_generated = True

    # Step 1: Generate an initial summary
    initial_prompt_text = """
    You are a summarization expert. Your task is to write summaries.
    Here is the original document text:
    
    {text}
    
    Start by creating a summary in your first pass. 
    
    Create an initial summary.
    """
    initial_prompt = ChatPromptTemplate.from_template(initial_prompt_text)
    current_summary = model(initial_prompt.format(text=text))
    
    display(Markdown(f"### Initial Summary (Iteration {iteration})"))
    display(Markdown(current_summary))

    # Iterative refinement process
    while iteration < max_iterations and questions_generated:
        print("-----------------Next iteration-----------------")
        iteration += 1
        # Step 2: Ask the LLM to compare the original text and summary, and to refine it
        refinement_prompt_text = """
        You are a summarization expert. Your task is to refine summaries.
        Here is the original document text:
        
        {text}
        
        Refine the below current summary, keep it as it is but ensure it becomes more complete, coherent, clear, and accurate. 
        Aim to capture the essence of the text with each refinement.
        
        Current summary:
        {summary}
        
        Please provide a refined summary below:
        """
        refinement_prompt = ChatPromptTemplate.from_template(refinement_prompt_text)
        current_summary = model(refinement_prompt.format(text = text, 
                                                         summary = current_summary))
    
    return current_summary


In [53]:
document_text = extract_text_from_pdf("../documents/ey-eu-ai-act-political-agreement-overview-february-2024.pdf")

In [54]:
final_summary = generate_refined_summary(model, document_text)
print("\n\n\n\nFinal Summary:")
display(Markdown(final_summary))

### Initial Summary (Iteration 0)

Here's a concise summary of the text:

The EU AI Act is expected to be approved by the end of April 2024 and will come into force 20 days after publication, starting a phased implementation timeline. The Act sets guidelines for the development and use of artificial intelligence in the European Union. The next steps include international alignment with other countries through organizations like the Council of Europe, G7, and UN to promote compatible rules globally. Additionally, a voluntary AI Pact is planned to encourage industry commitment to implementing requirements ahead of legal deadlines.

Let me know if you'd like me to make any adjustments or expand on this summary!

-----------------Next iteration-----------------
-----------------Next iteration-----------------
-----------------Next iteration-----------------
-----------------Next iteration-----------------
-----------------Next iteration-----------------




Final Summary:


Here is a refined summary:

**European Union's Artificial Intelligence Act (AI Act)**: The EU's AI Act is a significant regulatory milestone expected to be finalized by April 2024. It establishes a phased implementation timeline starting after publication.

**Key Roles and Responsibilities**: The Act defines key roles, including Providers, Deployers, Authorized Representatives, Importers, Distributors, Product Manufacturers, and Operators. These entities will be held accountable for ensuring the safety, security, and transparency of AI systems.

**Requirements and Provisions**: The AI Act requires:

* **Transparency and Explainability**: Ensuring users understand how AI decisions are made.
* **Risk-Based Approach**: Identifying potential risks and mitigating them to ensure safety and security of AI systems.
* **Accountability and Liability**: Holding organizations responsible for any negative consequences related to AI.

**International Collaboration**: The EU Commission plans to collaborate with other countries through international organizations to promote alignment. A **Voluntary AI Pact** will encourage industry commitment to implementing requirements ahead of legal deadlines, fostering a culture of proactive compliance.

**Leadership in Responsible AI Governance**: The EU aims to establish itself as a leader in developing and promoting responsible AI governance globally, ensuring European citizens benefit from the safe and trustworthy use of AI. EY (Ernst & Young) supports this effort by highlighting their commitment to building trust and using data-driven insights to create a better working world.

This refined summary captures the essence of the original text while providing a more complete, coherent, clear, and accurate understanding of the EU's Artificial Intelligence Act.